## Building a BATTLE Eval

In [1016]:
file_name = "dataset"
model = "gpt-3.5-turbo"
choices = "user"                     # options: user, ai
num_few_shot = 4
initial_message = [{"role": "user", "content": ""}]
roles = "user"                      # options: system-user, system, user
battles_generation = "dense"        # options: dense, only-new

#!cd evals
#!git lfs fetch --all
#!git lfs pull

In [1017]:
import os
from openai import OpenAI
import pandas as pd
import yaml
from IPython.display import display_markdown

client = OpenAI()
#import datetime
#from dotenv import load_dotenv

#load_dotenv()  # Load environment variables from .env file

#api_key = os.environ.get("OPENAI_API_KEY")

# Install Evals if you haven't already
# %pip install -e ../.
# pip install --upgrade openai
# %pip install pandas


In [1018]:
# Paths. Assuming this notebook is in examples/

evals_path = os.path.join(os.getcwd(), "..", "evals")

registry_path = os.path.join(evals_path, "registry", "evals", "battles.yaml")

data_path = os.path.join(evals_path, "registry", "data", "battles")
os.makedirs(data_path, exist_ok=True)
data_path = os.path.join(data_path, "samples.jsonl")

json_logs_path = os.path.join(os.getcwd(), "logs")
os.makedirs(json_logs_path, exist_ok=True)
json_logs_path = os.path.join(json_logs_path, "logs")

df_path = os.path.join(evals_path, "evallogs", "df")
os.makedirs(df_path, exist_ok=True)
dataset_path = os.path.join(df_path, file_name)

In [1019]:
# Registry yaml

registry_yaml = {}

registry_yaml["battles"] = {
    "id": "battles.test.v1",
    "metrics": ["accuracy"]
}
registry_yaml["battles.test.v1"] = {
    "class": "evals.elsuite.modelgraded.classify:ModelBasedClassify",
    "args": {
        "samples_jsonl": "battles/samples.jsonl",
        "eval_type": "cot_classify",
        "modelgraded_spec": "battle"
    }
}
with open(os.path.join(registry_path), "w") as f:
    yaml.dump(registry_yaml, f)

In [1020]:
# Load data

try:
    with open(dataset_path, "r") as f:
        dataset = pd.read_json(f, lines=True)
    if dataset.iloc[-1]["Choice"] == "No":
        best_content = dataset.iloc[-1]["Instruction2"]
        best_response = dataset.iloc[-1]["Response2"]
    else:
        best_content = dataset.iloc[-1]["Instruction1"]
        best_response = dataset.iloc[-1]["Response1"]
    best_message = [{"role": "user", "content": best_content}]

# Or create new data file

except FileNotFoundError:
    dataset = []
    dataset = pd.DataFrame(dataset, columns=['Instruction1', 'Response1', 'Instruction2', 'Response2', 'Sampled', 'Choice', 'Data'])
    best_message = initial_message
    best_response = client.chat.completions.create(
        messages = initial_message,
        model = model,
        ).choices[0].message.content
    
candidate_messages = [best_message]
next_candidate_messages = []
generation_distance = 0
message_in_candidate_messages = 0


In [1022]:
def battle(best_message, best_response, candidate_message):

    dataset = [{"input1": candidate_message, "input2": best_message, "completion2":best_response}]

    df = pd.DataFrame(dataset)
    df.to_json(data_path, orient="records", lines=True)

    !oaieval gpt-3.5-turbo battles --record_path logs/logs
    
    with open(json_logs_path, "r") as f:
        df = pd.read_json(f, lines=True)

    #current_time = datetime.datetime.now()
    #formatted_time = current_time.strftime("%Y%m%d_%H%M%S")
    #df.to_json(os.path.join(df_path, formatted_time), lines=True, orient="records")

    instruction1 = candidate_message[-1]["content"]
    instruction2 = best_message[-1]["content"]

    battle_prompt_content = df["data"].iloc[-2]["prompt"][0]["content"]
    response1 = battle_prompt_content.split("\n[Response 1]\n",)[1].split("\n\n[Instruction 2]\n")[0]
    response1 = response1.replace("\\'", "'").replace("\\n", "\n")
    response2 = battle_prompt_content.split("\n[Response 2]\n",)[1].split("\n\n\nIs the first response better than the second?")[0]
    response2 = response2.replace("\\'", "'").replace("\\n", "\n")
    print(f"Response1: {response1}")
    #print(f"response2: {response2}")

    sampled = df["data"].iloc[-2]["sampled"][0]

    choice = df["data"].iloc[-1]["choice"]

    data = {'Instruction1': instruction1, 'Response1': response1, 'Instruction2': instruction2, 'Response2': response2, 'Sampled': sampled, 'Choice': choice, 'Data': {}}
    data = pd.DataFrame([data])
    data.at[0, "Data"] = df.to_dict()
    return data

In [1023]:
def switch_system_and_user(message):
    new_message = []
    if message[0]["role"] == "system":
        new_message.append([{"role": "user", "content": message[0]["content"]}])
    else:
        new_message.append([{"role": "system", "content": message[0]["content"]}])
    return new_message

In [1024]:
def find_parents(content, dataset):
    #print (f"finding parents {content}")
    dataset = pd.DataFrame(dataset)
    dataset = dataset[dataset["Response1"] == content]
    parents = dataset["Instruction1"].unique()
    return parents

In [1025]:
def find_children(content, dataset):
    #print (f"finding children {content}")
    dataset = pd.DataFrame(dataset)
    dataset = dataset[dataset["Instruction1"] == content]
    children = dataset["Response1"].unique()
    return children

In [1026]:
def list_candidate_messages(dataset, best_message, generation_distance, roles):
    best_content = best_message[-1]["content"]
    last = [best_content]
    list_of_contents = [best_content]

    next = []
    for i in range(generation_distance):
        #print (f"starting level {i}, generation distance {generation_distance}")
        for content in last:
            #print(f"last_up: {last_up}, now {content}")
            next.extend(find_parents(content, dataset))
            next.extend(find_children(content, dataset))
            
        #print (f"behind for loops")
        list_of_contents.extend(next)
        last = next.copy()
        #print(f"endind level {i}")
        
    list_of_contents = pd.array(list_of_contents).unique().tolist()
    
    #best_message_index = list_of_contents.index(best_message[0][content])
    #start = max(best_message_index - generation_distance, 0)
    #stop = min(len(list_of_contents))
    #stop = len(list_of_contents) - 1

    messages = []
    for content in list_of_contents:
        if roles == "system-user" or "user":
            messages.append([{"role":"user","content":content}])
        if (roles == "system-user") or (roles == "system"):
            print("Creating message without user role.")
            messages.append([{"role":"system","content":content}])

    return messages

In [1047]:
def make_x_shot_prompt(dataset,message,num_few_shot):
    few_shot_prompt = []
    for _, key_battle in dataset[dataset["Choice"]=="Yes"].iloc[-num_few_shot:].iterrows():
        few_shot_prompt.append({"role": "system", "content": key_battle["Instruction1"], "name": "example_user"})
        few_shot_prompt.append({"role": "system", "content": key_battle["Response1"], "name": "example_assistant"})
    message = few_shot_prompt + message
    return message


In [ ]:
if choices == "user":

    prompt = "You are comparing two responses to the following two instructions."

    prompt += "\n\n[Instruction 1]\n"
    prompt += candidate_message[-1]["content"]

    prompt += "\n\n[Response 1]\n"
    if message_in_candidate_messages <= len(candidate_messages):
        candidate_message = candidate_messages[message_in_candidate_messages]
        message_in_candidate_messages += 1
    else:
        if battles_generation == "dense":
            candidate_messages = list_candidate_messages(dataset, best_message, generation_distance, roles)
            generation_distance += 1
            candidate_message = candidate_messages[0]
            message_in_candidate_messages = 1
            print(f"candidate messages listed, generation distance: {generation_distance}, number of candidate messages: {len(candidate_messages)}")
        else:
            print("Code not completed!")
    completion = client.chat.completions.create(
        messages = candidate_message,
        model = model,
        )
    response1 = completion.choices[0].message.content
    prompt += response1
    
    prompt += "\n\n[Instruction 2]\n"
    prompt += best_message[-1]["content"]

    prompt += "\n\n[Response 2]\n"
    prompt += best_response

    prompt += "\n\n\nIs the first response better than the second? You must provide one answer based on your subjective view."

    data = {'Instruction1': candidate_message[-1]["content"], 'Response1': response1, 'Instruction2': best_message[-1]["content"], 'Response2': best_response, 'Sampled': {}, 'Choice': "", 'Data': {}}
    data = pd.DataFrame([data])
    data.at[0, "Data"] = completion.to_dict()

    display_markdown(prompt, raw=True)

In [ ]:
choice = "No"
#choice = "Yes"
data.at[0, "Data"] = choice
if choice == "Yes":
    best_message = candidate_message
    best_response = data["Response1"].iloc[0]
    print(f"New best message and response!")
    if battles_generation == "dense":
        generation_distance = 0
dataset = pd.concat([dataset, data],ignore_index=True)
dataset.to_json(os.path.join(df_path, "dataset"), lines=True, orient="records")
        

In [1053]:
if choices == "ai":
    for candidate_message in candidate_messages:
        if num_few_shot > 0:
            candidate_message = make_x_shot_prompt(dataset,candidate_message,num_few_shot)
        data = battle(best_message, best_response, candidate_message)
        dataset = pd.concat([dataset, data],ignore_index=True)
        dataset.to_json(os.path.join(df_path, "dataset"), lines=True, orient="records")
        if data["Choice"].iloc[0] == "Yes":
            best_message = candidate_message
            best_response = data["Response1"].iloc[0]
            print(f"New best message:{best_message}")
            if battles_generation == "dense":
                generation_distance = 0

        print(f"Instruction1: {candidate_message[-1]["content"]}")

        if battles_generation == "only-new":
            new_message = [{"role": candidate_message[0]["role"], "content":data["Response1"].iloc[0]}]
            if new_message not in next_candidate_messages:
                next_candidate_messages.append(new_message)
                if roles == "system-user":
                    next_candidate_messages.append(switch_system_and_user(new_message))

    if battles_generation == "only-new":
        candidate_messages = next_candidate_messages
        next_candidate_messages = []
    if battles_generation == "dense":
        #print("going to list candidate messages")
        candidate_messages = list_candidate_messages(dataset, best_message, generation_distance, roles)
        generation_distance += 1
        #print("candidate messages listed")
    print(f"all done, generation distance: {generation_distance}, number of candidate messages: {len(candidate_messages)}")

[2024-08-10 10:23:15,386] [registry.py:271] Loading registry from /Users/janvotava/Desktop/evals/evals/registry/evals
[2024-08-10 10:23:16,645] [registry.py:271] Loading registry from /Users/janvotava/.evals/evals
[2024-08-10 10:23:16,647] [oaieval.py:215] Run started: 24081008231646NWPOLA
[2024-08-10 10:23:16,649] [registry.py:271] Loading registry from /Users/janvotava/Desktop/evals/evals/registry/modelgraded
[2024-08-10 10:23:16,674] [registry.py:271] Loading registry from /Users/janvotava/.evals/modelgraded
[2024-08-10 10:23:16,674] [data.py:94] Fetching /Users/janvotava/Desktop/evals/evals/registry/data/battles/samples.jsonl
[2024-08-10 10:23:16,675] [eval.py:36] Evaluating 1 samples
[2024-08-10 10:23:16,688] [eval.py:144] Running in threaded mode with 10 threads!
100%|█████████████████████████████████████████████| 1/1 [00:02<00:00,  2.62s/it]
[2024-08-10 10:23:19,313] [oaieval.py:275] Found 2/2 sampling events with usage data
[2024-08-10 10:23:19,313] [oaieval.py:283] Token usage

In [1049]:
for entry in dataset[dataset["Choice"]=="Yes"].iloc[-1].drop("Data"):
    display_markdown(entry, raw=True)

Hello! I'm here to help with any questions or tasks you may have. Just let me know how I can assist you.

Thank you for offering your help! If you have any questions or need assistance with anything, feel free to ask. I'm here to help you.

Hello! I'm here to help with any questions or tasks you may have. Just let me know how I can assist you.

Hello! How can I assist you today?

1. The first response acknowledges the offer of help and expresses gratitude, creating a positive interaction.
2. The first response also reiterates the willingness to assist and encourages the user to ask for help if needed.
3. The second response is more direct and simply asks how the assistant can assist without acknowledging the initial offer of help.

Based on the above points, the first response is better than the second.

Yes

Yes

In [1050]:
dataset.iloc[-20:]

,Instruction1,Response1,Instruction2,Response2,Sampled,Choice,Data
0,,Hello! How can I assist you today?,,Hello! How can I assist you today?,1. Both responses are identical in content and...,No,{'spec': {0: {'completion_fns': ['gpt-3.5-turb...
1,,Hello! How can I assist you today?,,Hello! How can I assist you today?,1. Both responses are identical in content and...,No,{'spec': {0: {'completion_fns': ['gpt-3.5-turb...
2,,Hello! How can I assist you today?,,Hello! How can I assist you today?,1. Both responses are identical in content and...,No,{'spec': {0: {'completion_fns': ['gpt-3.5-turb...
3,Hello! How can I assist you today?,Hello! I'm here to help with any questions or ...,,Hello! How can I assist you today?,1. Response 1 provides a more personalized and...,No,{'spec': {0: {'completion_fns': ['gpt-3.5-turb...
4,,Hello! How can I assist you today?,,Hello! How can I assist you today?,1. Both responses are identical in content and...,No,{'spec': {0: {'completion_fns': ['gpt-3.5-turb...
5,Hello! How can I assist you today?,Hello! I'm here to help with any questions or ...,,Hello! How can I assist you today?,1. Response 1 provides a more detailed and pro...,No,{'spec': {0: {'completion_fns': ['gpt-3.5-turb...
6,Hello! I'm here to help with any questions or ...,Hello! How can I assist you today?,,Hello! How can I assist you today?,1. Both responses start with a friendly greeti...,No,{'spec': {0: {'completion_fns': ['gpt-3.5-turb...
7,,Hello! How can I assist you today?,,Hello! How can I assist you today?,1. Both responses are identical in content and...,No,{'spec': {0: {'completion_fns': ['gpt-3.5-turb...
8,Hello! How can I assist you today?,Hello! I'm here to help with any questions or ...,,Hello! How can I assist you today?,1. Response 1 provides a more proactive and he...,No,{'spec': {0: {'completion_fns': ['gpt-3.5-turb...
9,Hello! I'm here to help with any questions or ...,Hello! How can I assist you today?,,Hello! How can I assist you today?,1. Both responses start with a friendly greeti...,Yes,{'spec': {0: {'completion_fns': ['gpt-3.5-turb...


In [1051]:
candidate_messages

[[{'role': 'user',
   'content': "Hello! I'm here to help with any questions or tasks you may have. Just let me know how I can assist you."}],
 [{'role': 'user', 'content': 'Hello! How can I assist you today?'}],
 [{'role': 'user',
   'content': "Thank you for offering your help! If you have any questions or need assistance with anything, feel free to ask. I'm here to help you."}]]

In [1055]:
dataset["Sampled"].unique()

array(['1. Both responses are identical in content and structure.\n2. There is no distinguishable difference between the two responses.\n3. Both responses follow the same format and tone.\n\nNo\n\nNo',
       '1. Both responses are identical in content and structure.\n2. There is no distinguishable difference between the two responses.\n3. Therefore, neither response is better than the other.\n\nNo\n\nNo',
       '1. Response 1 provides a more personalized and proactive approach by stating that the person is there to help with any questions or tasks.\n2. Response 2 is more generic and does not offer any specific assistance right away.\n\nNo\n\nNo',
       '1. Response 1 provides a more detailed and proactive offer of assistance compared to Response 2, which is more generic.\n2. Response 1 shows a willingness to help with any questions or tasks, making it more customer-focused.\n3. Response 2 is a bit more abrupt and lacks the same level of engagement as Response 1.\n\nNo\nNo',
       '